In [1]:
import numpy as np
import pandas as pd
import random

### Death: Florida and its reference states

In [2]:
all_death = pd.read_csv('../20_intermediate_files/only_od_all_years.csv')
all_death.head()

,County,County Code,Year,Year Code,Drug/Alcohol Induced Cause,Drug/Alcohol Induced Cause Code,Deaths,CTYNAME,STNAME
0,"Cochise County, AZ",4003.0,2007.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,13.0,Cochise County,AZ
1,"Gila County, AZ",4007.0,2007.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,10.0,Gila County,AZ
2,"Maricopa County, AZ",4013.0,2007.0,2007.0,Drug poisonings (overdose) Unintentional (X40-...,D1,411.0,Maricopa County,AZ
3,"Maricopa County, AZ",4013.0,2007.0,2007.0,Drug poisonings (overdose) Suicide (X60-X64),D2,118.0,Maricopa County,AZ
4,"Maricopa County, AZ",4013.0,2007.0,2007.0,Drug poisonings (overdose) Undetermined (Y10-Y14),D4,45.0,Maricopa County,AZ


In [3]:
fl_pop = pd.read_csv('../20_intermediate_files/fl_death_pop.csv')
fl_pop.STNAME.unique()

array(['Colorado', 'Florida', 'Louisiana', 'Nevada'], dtype=object)

In [4]:
# select only florida and its reference states
fl_ref_states = ['LA', 'NV', 'CO', 'FL']
fl_death = all_death[all_death['STNAME'].isin(fl_ref_states)]

# rename the abbrevation state name to its full name
fl_death_ = fl_death.replace({'NV': 'Nevada', 'FL': 'Florida', 'LA': 'Louisiana', 'CO': 'Colorado'})

# make sure the state names are the same
assert (fl_death_['STNAME'].unique() == fl_pop['STNAME'].unique()).all()

# rename year
fl_death_ = fl_death_.rename(columns={'Year': 'YEAR'})

# remove useless columns: County, Year Code
fl_death_ = fl_death_.loc[:, [ 'YEAR', 'STNAME', 'CTYNAME','Deaths']]

In [5]:
# Collapsing the dataset to get total number of deaths per county per year
fl_agg_death = fl_death_.groupby(['YEAR', 'STNAME', 'CTYNAME'], as_index=False)['Deaths'].sum()
fl_agg_death.head()

,YEAR,STNAME,CTYNAME,Deaths
0,2003.0,Colorado,Adams County,42.0
1,2003.0,Colorado,Arapahoe County,45.0
2,2003.0,Colorado,Boulder County,20.0
3,2003.0,Colorado,Denver County,107.0
4,2003.0,Colorado,El Paso County,53.0


In [6]:
# merge the two dataframes
fl_death_pop = pd.merge(fl_pop, fl_agg_death, how='outer', on = ['STNAME', 'CTYNAME', 'YEAR'], indicator=True)
fl_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge
0,Colorado,Adams County,2000,350961,NaN,left_only
1,Colorado,Alamosa County,2000,14964,NaN,left_only
2,Colorado,Arapahoe County,2000,491701,NaN,left_only
3,Colorado,Archuleta County,2000,10038,NaN,left_only
4,Colorado,Baca County,2000,4501,NaN,left_only


In [7]:
 # make sure the number of rows is correct
 assert fl_death_pop.shape[0] == fl_pop.shape[0]
 assert fl_death_pop[fl_death_pop["_merge"] == 'both'].shape[0] == fl_agg_death.shape[0]

In [8]:
# remove the year where we don't have the opoid death data
min_year = fl_death_['YEAR'].min()
max_year = fl_death_['YEAR'].max()

fl_death_pop = fl_death_pop[(fl_death_pop['YEAR'] >= min_year) & (fl_death_pop['YEAR'] <= max_year)].reset_index(drop=True)
fl_death_pop

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge
0,Colorado,Adams County,2003,378785,42.0,both
1,Colorado,Alamosa County,2003,15125,NaN,left_only
2,Colorado,Arapahoe County,2003,514406,45.0,both
3,Colorado,Archuleta County,2003,11167,NaN,left_only
4,Colorado,Baca County,2003,4106,NaN,left_only
...,...,...,...,...,...,...
2751,Nevada,Pershing County,2015,6620,NaN,left_only
2752,Nevada,Storey County,2015,3881,NaN,left_only
2753,Nevada,Washoe County,2015,442617,84.0,both
2754,Nevada,White Pine County,2015,9875,NaN,left_only


In [9]:
# Calculate the death rate
fl_death_pop['Death Rate (%)'] = 100 * fl_death_pop['Deaths'] / fl_death_pop['POPULATION']
fl_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge,Death Rate (%)
0,Colorado,Adams County,2003,378785,42.0,both,0.011088
1,Colorado,Alamosa County,2003,15125,NaN,left_only,NaN
2,Colorado,Arapahoe County,2003,514406,45.0,both,0.008748
3,Colorado,Archuleta County,2003,11167,NaN,left_only,NaN
4,Colorado,Baca County,2003,4106,NaN,left_only,NaN


In [10]:
# missing values
fl_death_pop.isna().sum()

STNAME               0
CTYNAME              0
YEAR                 0
POPULATION           0
Deaths            1933
_merge               0
Death Rate (%)    1933
dtype: int64

We only have missing values for deaths

In [11]:
# use each year each state's mean to impute the missing values
imputed_fl_death_pop = fl_death_pop.loc[:, ['YEAR', 'STNAME', 'CTYNAME', 'Death Rate (%)', 'POPULATION']]
imputed_fl_death_pop['Death Rate (%)'] = imputed_fl_death_pop.loc[:, ['YEAR', 'STNAME', 'Death Rate (%)']].groupby(['YEAR', 'STNAME'], as_index=False).transform(lambda x: x.fillna(x.mean()))

In [12]:
# check whether there is any missing value
assert imputed_fl_death_pop.isna().sum().sum() == 0

In [13]:
# Add indicator for treatment and control group
imputed_fl_death_pop['Indicator'] = imputed_fl_death_pop['STNAME'].apply(lambda x: "Treatment" if x == 'Florida' else "Control")
imputed_fl_death_pop.head()

,YEAR,STNAME,CTYNAME,Death Rate (%),POPULATION,Indicator
0,2003,Colorado,Adams County,0.011088,378785,Control
1,2003,Colorado,Alamosa County,0.009527,15125,Control
2,2003,Colorado,Arapahoe County,0.008748,514406,Control
3,2003,Colorado,Archuleta County,0.009527,11167,Control
4,2003,Colorado,Baca County,0.009527,4106,Control


In [14]:
# write to csv
imputed_fl_death_pop.to_csv('../20_intermediate_files/florida_death_cleaned.csv', index=False)

### Death: Texas and its reference states

In [15]:
tx_pop = pd.read_csv('../20_intermediate_files/tx_death_pop.csv')
tx_pop.STNAME.unique()

array(['New York', 'Oregon', 'Texas', 'Wisconsin'], dtype=object)

In [16]:
# select only texas and its reference states
tx_ref_states = ['NY', 'OR', 'TX', 'WI']
tx_death = all_death[all_death['STNAME'].isin(tx_ref_states)]

# rename the abbrevation state name to its full name
tx_death_ = tx_death.replace({'NY': 'New York', 'OR': 'Oregon', 'TX': 'Texas', 'WI': 'Wisconsin'})

# make sure the state names are the same
assert (tx_death_['STNAME'].unique() == tx_pop['STNAME'].unique()).all()

# rename year
tx_death_ = tx_death_.rename(columns={'Year': 'YEAR'})

# remove useless columns: County, Year Code
tx_death_ = tx_death_.loc[:, [ 'YEAR', 'STNAME', 'CTYNAME','Deaths']]

In [17]:
# Collapsing the dataset to get total number of deaths per county per year
tx_agg_death = tx_death_.groupby(['YEAR', 'STNAME', 'CTYNAME'], as_index=False)['Deaths'].sum()
tx_agg_death.head()

,YEAR,STNAME,CTYNAME,Deaths
0,2003.0,New York,Bronx County,131.0
1,2003.0,New York,Kings County,216.0
2,2003.0,New York,Nassau County,21.0
3,2003.0,New York,New York County,197.0
4,2003.0,New York,Onondaga County,10.0


In [18]:
# merge the two dataframes
tx_death_pop = pd.merge(tx_pop, tx_agg_death, how='outer', on = ['STNAME', 'CTYNAME', 'YEAR'], indicator=True)
tx_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge
0,New York,Albany County,2000,294960,NaN,left_only
1,New York,Allegany County,2000,49916,NaN,left_only
2,New York,Bronx County,2000,1333854,NaN,left_only
3,New York,Broome County,2000,200319,NaN,left_only
4,New York,Cattaraugus County,2000,84017,NaN,left_only


In [19]:
 # make sure the number of rows is correct
 assert tx_death_pop.shape[0] == tx_pop.shape[0]
 assert tx_death_pop[tx_death_pop["_merge"] == 'both'].shape[0] == tx_agg_death.shape[0]

In [20]:
# remove the year where we don't have the opoid death data
min_year = tx_death_['YEAR'].min()
max_year = tx_death_['YEAR'].max()

tx_death_pop = tx_death_pop[(tx_death_pop['YEAR'] >= min_year) & (tx_death_pop['YEAR'] <= max_year)].reset_index(drop=True)
tx_death_pop

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge
0,New York,Albany County,2003,298491,NaN,left_only
1,New York,Allegany County,2003,50220,NaN,left_only
2,New York,Bronx County,2003,1353712,131.0,both
3,New York,Broome County,2003,198364,NaN,left_only
4,New York,Cattaraugus County,2003,82968,NaN,left_only
...,...,...,...,...,...,...
5507,Wisconsin,Waukesha County,2015,396377,34.0,both
5508,Wisconsin,Waupaca County,2015,51684,NaN,left_only
5509,Wisconsin,Waushara County,2015,23966,NaN,left_only
5510,Wisconsin,Winnebago County,2015,169383,22.0,both


In [21]:
# Calculate the death rate
tx_death_pop['Death Rate (%)'] = 100 * tx_death_pop['Deaths'] / tx_death_pop['POPULATION']
tx_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge,Death Rate (%)
0,New York,Albany County,2003,298491,NaN,left_only,NaN
1,New York,Allegany County,2003,50220,NaN,left_only,NaN
2,New York,Bronx County,2003,1353712,131.0,both,0.009677
3,New York,Broome County,2003,198364,NaN,left_only,NaN
4,New York,Cattaraugus County,2003,82968,NaN,left_only,NaN


In [22]:
# missing values
tx_death_pop.isna().sum()

STNAME               0
CTYNAME              0
YEAR                 0
POPULATION           0
Deaths            4587
_merge               0
Death Rate (%)    4587
dtype: int64

We only have missing values for deaths

In [23]:
# use each year each state's mean to impute the missing values
imputed_tx_death_pop = tx_death_pop.loc[:, ['YEAR', 'STNAME', 'CTYNAME', 'Death Rate (%)', 'POPULATION']]
imputed_tx_death_pop['Death Rate (%)'] = imputed_tx_death_pop.loc[:, ['YEAR', 'STNAME', 'Death Rate (%)']].groupby(['YEAR', 'STNAME'], as_index=False).transform(lambda x: x.fillna(x.mean()))

In [24]:
# check whether there is any missing value
assert imputed_tx_death_pop.isna().sum().sum() == 0

In [25]:
# Add indicator for treatment and control group
imputed_tx_death_pop['Indicator'] = imputed_tx_death_pop['STNAME'].apply(lambda x: "Treatment" if x == 'Texas' else "Control")
imputed_tx_death_pop.head()

,YEAR,STNAME,CTYNAME,Death Rate (%),POPULATION,Indicator
0,2003,New York,Albany County,0.005506,298491,Control
1,2003,New York,Allegany County,0.005506,50220,Control
2,2003,New York,Bronx County,0.009677,1353712,Control
3,2003,New York,Broome County,0.005506,198364,Control
4,2003,New York,Cattaraugus County,0.005506,82968,Control


In [26]:
# write to csv
imputed_tx_death_pop.to_csv('../20_intermediate_files/texas_death_cleaned.csv', index=False)

### Death: Washington and its reference states

In [27]:
wa_pop = pd.read_csv('../20_intermediate_files/wa_death_pop.csv')
wa_pop.STNAME.unique()

array(['Hawaii', 'Oklahoma', 'Oregon', 'Washington'], dtype=object)

In [28]:
# select only washington and its reference states
wa_ref_states = ['HI', 'OK', 'OR', 'WA']
wa_death = all_death[all_death['STNAME'].isin(wa_ref_states)]

# rename the abbrevation state name to its full name
wa_death_ = wa_death.replace({'HI': 'Hawaii', 'OK': 'Oklahoma', 'OR': 'Oregon', 'WA': 'Washington'})

# make sure the state names are the same
assert (wa_death_['STNAME'].unique() == wa_pop['STNAME'].unique()).all()

# rename year
wa_death_ = wa_death_.rename(columns={'Year': 'YEAR'})

# remove useless columns: County, Year Code
wa_death_ = wa_death_.loc[:, [ 'YEAR', 'STNAME', 'CTYNAME','Deaths']]

In [29]:
# Collapsing the dataset to get total number of deaths per county per year
wa_agg_death = wa_death_.groupby(['YEAR', 'STNAME', 'CTYNAME'], as_index=False)['Deaths'].sum()
wa_agg_death.head()

,YEAR,STNAME,CTYNAME,Deaths
0,2003.0,Hawaii,Hawaii County,11.0
1,2003.0,Hawaii,Honolulu County,62.0
2,2003.0,Oklahoma,Cleveland County,15.0
3,2003.0,Oklahoma,Oklahoma County,75.0
4,2003.0,Oklahoma,Tulsa County,100.0


In [30]:
# merge the two dataframes
wa_death_pop = pd.merge(wa_pop, wa_agg_death, how='outer', on = ['STNAME', 'CTYNAME', 'YEAR'], indicator=True)
wa_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge
0,Hawaii,Hawaii County,2000,149095,NaN,left_only
1,Hawaii,Honolulu County,2000,875061,NaN,left_only
2,Hawaii,Kalawao County,2000,145,NaN,left_only
3,Hawaii,Kauai County,2000,58511,NaN,left_only
4,Hawaii,Maui County,2000,128754,NaN,left_only


In [31]:
 # make sure the number of rows is correct
 assert wa_death_pop.shape[0] == wa_pop.shape[0]
 assert wa_death_pop[wa_death_pop["_merge"] == 'both'].shape[0] == wa_agg_death.shape[0]

In [32]:
# remove the year where we don't have the opoid death data
min_year = wa_death_['YEAR'].min()
max_year = wa_death_['YEAR'].max()

wa_death_pop = wa_death_pop[(wa_death_pop['YEAR'] >= min_year) & (wa_death_pop['YEAR'] <= max_year)].reset_index(drop=True)
wa_death_pop

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge
0,Hawaii,Hawaii County,2003,156340,11.0,both
1,Hawaii,Honolulu County,2003,888026,62.0,both
2,Hawaii,Kalawao County,2003,129,NaN,left_only
3,Hawaii,Kauai County,2003,60061,NaN,left_only
4,Hawaii,Maui County,2003,134742,NaN,left_only
...,...,...,...,...,...,...
2036,Washington,Wahkiakum County,2015,3989,NaN,left_only
2037,Washington,Walla Walla County,2015,59970,NaN,left_only
2038,Washington,Whatcom County,2015,211942,23.0,both
2039,Washington,Whitman County,2015,48224,NaN,left_only


In [33]:
# Calculate the death rate
wa_death_pop['Death Rate (%)'] = 100 * wa_death_pop['Deaths'] / wa_death_pop['POPULATION']
wa_death_pop.head()

,STNAME,CTYNAME,YEAR,POPULATION,Deaths,_merge,Death Rate (%)
0,Hawaii,Hawaii County,2003,156340,11.0,both,0.007036
1,Hawaii,Honolulu County,2003,888026,62.0,both,0.006982
2,Hawaii,Kalawao County,2003,129,NaN,left_only,NaN
3,Hawaii,Kauai County,2003,60061,NaN,left_only,NaN
4,Hawaii,Maui County,2003,134742,NaN,left_only,NaN


In [34]:
# missing values
wa_death_pop.isna().sum()

STNAME               0
CTYNAME              0
YEAR                 0
POPULATION           0
Deaths            1571
_merge               0
Death Rate (%)    1571
dtype: int64

We only have missing values for deaths

In [35]:
# use each year each state's mean to impute the missing values
imputed_wa_death_pop = wa_death_pop.loc[:, ['YEAR', 'STNAME', 'CTYNAME', 'Death Rate (%)', 'POPULATION']]
imputed_wa_death_pop['Death Rate (%)'] = imputed_wa_death_pop.loc[:, ['YEAR', 'STNAME', 'Death Rate (%)']].groupby(['YEAR', 'STNAME'], as_index=False).transform(lambda x: x.fillna(x.mean()))

In [36]:
# check whether there is any missing value
assert imputed_wa_death_pop.isna().sum().sum() == 0

In [37]:
# Add indicator for treatment and control group
imputed_wa_death_pop['Indicator'] = imputed_wa_death_pop['STNAME'].apply(lambda x: "Treatment" if x == 'Washington' else "Control")
imputed_wa_death_pop.head()

,YEAR,STNAME,CTYNAME,Death Rate (%),POPULATION,Indicator
0,2003,Hawaii,Hawaii County,0.007036,156340,Control
1,2003,Hawaii,Honolulu County,0.006982,888026,Control
2,2003,Hawaii,Kalawao County,0.007009,129,Control
3,2003,Hawaii,Kauai County,0.007009,60061,Control
4,2003,Hawaii,Maui County,0.007009,134742,Control


In [38]:
# write to csv
imputed_wa_death_pop.to_csv('../20_intermediate_files/washington_death_cleaned.csv', index=False)

Bad pipe message: %s [b'\x02\xe8\xf2o~\xc0T\xf1M/\x85\xa7\x1f{\xa60[\xda \xef\x9b\x0b-\x15\xd2\x97\xfb\xdb\xa1\xea\xe4KV', b'\xd6\xa8y\xf6`\xb8\xd5V\xd3n\x96\xaf\xe2.\x14u\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08', b'\x06\x04\x01\x05']
Bad pipe message: %s [b'\xad\x0e\xa0@\x94\xb9\xe45']
Bad pipe message: %s [b"V\xb8\x8c?\xd5v\xda\xc4\xb1O.\xb3\x8fH\x0b\x83oV\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x00